1. 1. Please create dimension tables dim_user , dim_post , and dim_date to store
normalized data from the raw tables


In [ ]:
-- Create dimension table dim_user
CREATE TABLE dim_user (
    user_id SERIAL PRIMARY KEY,
    user_name VARCHAR(100),
    country VARCHAR(50)
);

-- Create dimension table dim_post
CREATE TABLE dim_post (
    post_id SERIAL PRIMARY KEY,
    post_text VARCHAR(500),
    post_date DATE,
    user_id INT REFERENCES dim_user(user_id)
);

-- Create dimension table dim_date
CREATE TABLE dim_date (
    date_id SERIAL PRIMARY KEY,
    calendar_date DATE,
    day_of_week INT,
    month INT,
    year INT
);


2. Populate the dimension tables by inserting data from the related raw tables


In [ ]:
-- Populate dim_user from raw_users
INSERT INTO dim_user (user_name, country)
SELECT DISTINCT user_name, country FROM raw_users;

-- Populate dim_post from raw_posts
INSERT INTO dim_post (post_text, post_date, user_id)
SELECT DISTINCT post_text, post_date, user_id FROM raw_posts;

-- Populate dim_date from raw_posts
INSERT INTO dim_date (calendar_date, day_of_week, month, year)
SELECT DISTINCT post_date, EXTRACT(DOW FROM post_date), EXTRACT(MONTH FROM post_date), EXTRACT(YEAR FROM post_date) FROM raw_posts;


3. Create a fact table called fact_post_performance to store metrics like post views and
likes over time

In [ ]:
-- Create fact table fact_post_performance
CREATE TABLE fact_post_performance (
    fact_id SERIAL PRIMARY KEY,
    post_id INT REFERENCES dim_post(post_id),
    date_id INT REFERENCES dim_date(date_id),
    views INT,
    likes INT
);


4. Populate the fact table by joining and aggregating data from the raw tables

In [ ]:
-- Populate fact_post_performance from raw_likes
INSERT INTO fact_post_performance (post_id, date_id, likes)
SELECT post_id, d.date_id, COUNT(*) AS likes
FROM raw_likes l
JOIN dim_post p ON l.post_id = p.post_id
JOIN dim_date d ON l.like_date = d.calendar_date
GROUP BY post_id, d.date_id;

-- Update views in fact_post_performance
UPDATE fact_post_performance
SET views = 1; -- Assuming each like counts as a view, adjust accordingly


5. Please create a fact_daily_posts table to capture the number of posts per user per
day

In [ ]:
-- Create fact table fact_daily_posts
CREATE TABLE fact_daily_posts (
    fact_id SERIAL PRIMARY KEY,
    user_id INT REFERENCES dim_user(user_id),
    date_id INT REFERENCES dim_date(date_id),
    posts_count INT
);


6. Also populate the fact table by joining and aggregating data from the raw tables

In [ ]:
-- Populate fact_daily_posts from raw_posts
INSERT INTO fact_daily_posts (user_id, date_id, posts_count)
SELECT user_id, d.date_id, COUNT(*) AS posts_count
FROM raw_posts p
JOIN dim_date d ON p.post_date = d.calendar_date
GROUP BY user_id, d.date_id;
